In [64]:
import os
from googleapiclient.discovery import build
import json
from kafka import KafkaProducer
from time import sleep

In [8]:
api_key = os.environ.get("YOUTUBE_KEY")

In [9]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [10]:
users = ['andro4all', 'AndroideReview']

In [11]:
def getPlaylistId(userName):
    
    request = youtube.channels().list(
    part = 'contentDetails', 
    forUsername = userName
    )
    try: 
        user = request.execute()
        return(user['items'][0]['contentDetails']['relatedPlaylists']['uploads'])
    except: 
        print(f'ERROR READING VIDEOS FROM: {userName}')


In [16]:
def getVideos(playlistId, numOfVideos=50, page='CDIQAA' ):
    request = youtube.playlistItems().list(
    part = 'contentDetails', 
    playlistId = playlistId,
    maxResults = numOfVideos,
    pageToken = page)
    try:
        videos = request.execute()
        for i in videos['items']: 
            print(i['contentDetails']['videoId'])
        return [vid['contentDetails']['videoId'] for vid in videos['items']]
    except: 
        print(f"ERROR READING SONGS FROM PAGE {page}")
        return []

In [76]:
def getData(videoId):
    request = youtube.videos().list(
    part = 'statistics, snippet',
    id = videoId )
    try: 
        metadata = request.execute()
        stats = metadata['items'][0]['statistics']
        title = metadata['items'][0]['snippet']['title']
        date = metadata['items'][0]['snippet']['publishedAt']
        channelId = metadata['items'][0]['snippet']['channelId']
        try: 
            tags = metadata['items'][0]['snippet']['tags']
        except:
            tags =[]
        return {'stats':stats, 'videoId': videoId, 'channelId': channelId, 'date': date, 'title': title, 'tags':tags}  
#         return(stats,date , title, tags)
    except:
        print("ERROR ------------------")
        return()

In [77]:
producer = KafkaProducer(bootstrap_servers=['localhost:9099'],
                         value_serializer=lambda x: 
                         json.dumps(x).encode('utf-8'))

In [142]:
pl = map(lambda x: getPlaylistId(x), users)

In [143]:
vids = map(lambda x: getVideos(x,5), pl)

In [144]:
for col in vids: 
    for vid in col: 
        print(vid)
        print(json.dumps(getData(vid), indent=4))
        producer.send('test', json.dumps(getData(vid)))
        sleep(1)

L3cT6K3_y4U
RvqiTB-sQvo
L5f539m-U20
_Dv9XuAKjUw
GerzygeyDhU
L3cT6K3_y4U
{
    "stats": {
        "viewCount": "75643",
        "likeCount": "3277",
        "dislikeCount": "181",
        "favoriteCount": "0",
        "commentCount": "804"
    },
    "videoId": "L3cT6K3_y4U",
    "channelId": "UCok_yhjwg4WSx3s_2Yh8ZjQ",
    "date": "2020-07-01T17:30:01Z",
    "title": "Sony Xperia 1 Mark II vs Huawei P40 Pro. A POR el REY en fotograf\u00eda",
    "tags": [
        "Topesdegama",
        "TDG",
        "Terminal",
        "Dispositivo",
        "huawei",
        "p40",
        "p40 pro",
        "app gallery",
        "emui",
        "leica",
        "sony",
        "xperia 1",
        "xperia 1 mark II",
        "mark 2",
        "alpha",
        "comparativa",
        "versus",
        "vs"
    ]
}
RvqiTB-sQvo
{
    "stats": {
        "viewCount": "56596",
        "likeCount": "2465",
        "dislikeCount": "72",
        "favoriteCount": "0",
        "commentCount": "132"
    },
    "